## Long Range Transportation Plan

In [1]:
import _harmonization_utils as harmonization_utils
import geopandas as gpd
import numpy as np
import pandas as pd
from calitp_data_analysis.sql import to_snakecase
from shared_utils import geography_utils, utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/home/jovyan/data-analyses/project_list/_sb1_utils.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migrati

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

### SANDAG
* https://www.arcgis.com/home/item.html?id=ba8b2a6247ef463cb7411f039959f3ee
* https://www.arcgis.com/apps/mapviewer/index.html?webmap=ba8b2a6247ef463cb7411f039959f3ee

In [3]:
columns_to_keep = [
    "project_title",
    "lead_agency",
    "project_year",
    "project_category",
    "grant_program",
    "project_description",
    "total_project_cost",
    "location",
    "city",
    "county",
    "data_source",
    "notes",
]

In [4]:
def create_notes(df, note_cols: list):
    prefix = "_"
    for column in note_cols:
        df[f"{prefix}{column}"] = df[column].astype(str)
    note_cols = [prefix + sub for sub in note_cols]

    # https://stackoverflow.com/questions/65532480/how-to-combine-column-names-and-values
    def combine_notes(x):
        return ", ".join([col + ": " + x[col] for col in note_cols])

    df["notes"] = df.apply(combine_notes, axis=1)
    df.notes = df.notes.str.replace("_", " ")

    return df

In [5]:
def harmonizing_lrtp(
    df,
    project_name_col: str,
    project_description_col: str,
    project_category_col: str,
    project_cost_col: str,
    location_col: str,
    county_col: str,
    city_col: str,
    project_year_col: str,
    phase_col: str,
    program: str,
    data_source: str,
    note_cols: list,
    cost_in_millions: bool = True,
):
    """
    Take a dataset and change the column names/types to
    the same names and formats.
    """
    rename_columns = {
        project_name_col: "project_title",
        phase_col: "phase",
        project_description_col: "project_description",
        project_category_col: "project_category",
        project_cost_col: "total_project_cost",
        location_col: "geometry",
        county_col: "county",
        city_col: "city",
        project_year_col: "project_year",
    }
    # Rename columns
    df = df.rename(columns=rename_columns)

    # Coerce cost/fund columns to right type
    cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
    for i in cost_columns:
        try:
            df[i] = (
                df[i]
                .str.replace("$", "")
                .str.replace(",", "")
                .apply(pd.to_numeric, errors="coerce")
                .fillna(0)
            )
        except:
            df[i] = df[i].apply(pd.to_numeric, errors="coerce").fillna(0)

    # Add MPO & grant program
    df["lead_agency"] = program
    df["grant_program"] = f"{program} {data_source}"

    # Add data source
    df["data_source"] = data_source

    # Divide cost columns by millions
    # If bool is set to True
    if cost_in_millions:
        for i in cost_columns:
            df[i] = df[i].divide(1_000_000)

    # Fill in any nulls
    # df = df.fillna(df.dtypes.replace({"float64": 0.0, "object": "None"}))

    # Create columns even if they don't exist, just to harmonize
    # before concatting.
    create_columns = [
        "county",
        "city",
        "notes",
        "project_year",
        "project_category",
        "phase",
    ]
    for column in create_columns:
        if column not in df:
            df[column] = "None"
    if "geometry" not in df:
        df["geometry"] = None
    # Create notes
    df = create_notes(df, note_cols)

    columns_to_keep = [
        "project_title",
        "lead_agency",
        "project_year",
        "project_category",
        "grant_program",
        "project_description",
        "phase",
        "total_project_cost",
        "geometry",
        "city",
        "county",
        "data_source",
        "notes",
    ]

    df = df[columns_to_keep]

    return df

In [6]:
def open_rest_server(url_pt_1: str, url_pt_2: str, layer_name: list):
    full_gdf = pd.DataFrame()
    for i in layer_name:
        gdf = to_snakecase(gpd.read_file(f"{url_pt_1}{i}{url_pt_2}"))
        gdf["layer_name"] = i
        full_gdf = pd.concat([full_gdf, gdf], axis=0)

    return full_gdf

In [7]:
def sandag_lrtp():
    sandag_layers_list = [
        "Complete_Corridors_Connectors_Intersections_Ramps_RP2021",
        "Complete_Corridors_Highway_RP2021",
        "Transit_Leap_RP2021",
        "Adopted_Regional_Bike_Network_RP2021",
        "Mobility_Hubs_and_Flexible_Fleets",
        "Complete_Corridors_Regional_Arterials",
        "Goods_Movement",
    ]

    sandag_url_pt1 = (
        "https://services1.arcgis.com/HG80xaIVT1z1OdO5/ArcGIS/rest/services/"
    )
    sandag_url_pt2 = "/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*+&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="
    sandag = open_rest_server(sandag_url_pt1, sandag_url_pt2, sandag_layers_list)
   
    return sandag

In [8]:
def harmonize_sandag():
    df = sandag_lrtp()
    df = harmonizing_lrtp(
        df,
        project_name_col="project_name",
        project_description_col="description",
        project_category_col="category",
        project_cost_col="cost2020m",
        location_col="geometry",
        county_col="",
        city_col="",
        project_year_col="phase",
        phase_col="status",
        program="SANDAG",
        data_source="LRTP",
        note_cols=[
            "aqc_2016_1",
            "aqc_2020_1",
            "pricmcp",
            "conncmcp",
            "layer_name",
            "corridor_i",
            "type_1",
            "existing",
            "limits",
            "description_1",
            "route",
            "routetype",
            "route_desc",
            "rp_2021_id",
            "rp_2021_id_1",
            "capital_cost___2020__millions",
        ],
        cost_in_millions=False,
    )

    return df

### MTC
* 133 projects but point and line map returns 240 ish rows.
* Projects that are in different locations can have the same name split by open year...
* I guess the geospatial projects are split into various phases while the regular pandas df is just the project in its entirety.
* https://www.planbayarea.org/2050-plan/final-plan-bay-area-2050/final-supplemental-reports/interactive-transportation-project-list
* https://opendata.mtc.ca.gov/datasets/MTC::2023-transportation-improvement-program-projects-point/about
* https://opendata.mtc.ca.gov/datasets/MTC::plan-bay-area-2050-transportation-projects-line/about
* https://data.bayareametro.gov/Projects/Plan-Bay-Area-2050-Transportation-Project-List/y7ka-jksz

In [9]:
def mtc_lrtp():
    # Open rest server data
    layer_list = ["planbayarea2050_rtp_line", "planbayarea2050_rtp_point"]
    url_pt_1 = "https://services3.arcgis.com/i2dkYWmb4wHvYPda/ArcGIS/rest/services/"
    url_pt_2 = "/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*+&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="
    gdf = open_rest_server(url_pt_1, url_pt_2, layer_list)

    # Open all MTC projects. Not all projects are mappable.
    all_projects_url = "https://data.bayareametro.gov/resource/y7ka-jksz.json?$query=SELECT%0A%20%20%60plan_strategy%60%2C%0A%20%20%60rtpid%60%2C%0A%20%20%60title%60%2C%0A%20%20%60scope%60%2C%0A%20%20%60open_period%60%2C%0A%20%20%60funding_millions_yoe%60%2C%0A%20%20%60county%60"
    all_projects = pd.read_json(all_projects_url)

    # Merge info from all projects to gdf
    gdf = pd.merge(
        gdf, all_projects, how="inner", left_on=["proj_title"], right_on=["title"]
    )

    gdf = gdf.drop(columns=["title"])

    # Find projects that are unmappable
    mtc_geometry_titles = set(gdf.proj_title.unique().tolist())
    mtc_all_projects_titles = set(all_projects.title.unique().tolist())
    missing_projects = list(mtc_all_projects_titles - mtc_geometry_titles)

    # Add back in the  unmappable projects
    all_projects = all_projects[all_projects.title.isin(missing_projects)].reset_index(
        drop=True
    )
    all_projects = all_projects.rename(columns={"title": "proj_title"})
    final = pd.concat([all_projects, gdf])

    # Correct typo
    final = final.rename({"mode detai": "mode_detail"})

    # Set geometry again
    final = final.set_geometry("geometry").set_crs(geography_utils.WGS84)
    return final

In [10]:
def harmonize_mtc():
    df = mtc_lrtp()
    df = harmonizing_lrtp(
        df,
        project_name_col="proj_title",
        project_description_col="subcompone",
        project_category_col="plan_strategy",
        project_cost_col="funding_millions_yoe",
        location_col="geometry",
        county_col="county",
        city_col="",
        project_year_col="open_year",
        phase_col="",
        program="MTC",
        data_source="LRTP",
        note_cols=[
            "strategy",
            "layer_name",
            "mode_detai",
        ],
        cost_in_millions=False,
    )

    return df

### Tahoe
* Open data portal: https://www.tahoeopendata.org/search?tags=transportation
* https://www.tahoeopendata.org/datasets/TRPA::2020-regional-transportation-plan-projects/about

In [11]:
def tmpo_lrtp():
    tahoe_url = "https://maps.trpa.org/server/rest/services/Datadownloader_Transportation/MapServer/19/query?where=1%3D1&text=&objectIds=&time=&timeRelation=esriTimeRelationOverlaps&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=*+&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&returnExtentOnly=false&sqlFormat=none&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=geojson"
    gdf = to_snakecase(gpd.read_file(tahoe_url))
    gdf.complete_year = gdf.complete_year.astype(int)

    # Filter out projects that are not yet completed
    gdf2 = gdf[gdf.complete_year > 2022].reset_index(drop=True)

    gdf2["project_year"] = (
        gdf2.start_year.astype(str) + "-" + gdf.complete_year.astype(str)
    )
    return gdf2

In [12]:
def harmonize_tahoe():
    df = tmpo_lrtp()
    df = harmonizing_lrtp(
        df,
        project_name_col="project_name",
        project_description_col="description",
        project_category_col="category",
        project_cost_col="estimated_cost",
        location_col="geometry",
        county_col="",
        city_col="",
        project_year_col="project_year",
        program="TMPO",
        phase_col="phase",
        data_source="LRTP",
        note_cols=[
            "implementer",
            "funding_type",
            "plan_name",
            "label",
            "financial_status",
            "url",
        ],
        cost_in_millions=True,
    )

    return df

### Southern California Association of Governments (SCAG)

In [13]:
# LA 2016 landuse. just stuff like business parks/residential/etc
# scag1 = to_snakecase(gpd.read_file("https://services5.arcgis.com/YzWImMY4GtDcMxOx/ArcGIS/rest/services/2016_Land_Use_Information_for_Los_Angeles_County2/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*+&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="))

In [14]:
# Transit Priority Areas (Plan Year 2045) - SCAG region
# https://gisdata-scag.opendata.arcgis.com/datasets/4c80e5d1fa584c329676fbd23765edfc_0/explore?location=34.156350%2C-118.170650%2C9.63
#scag2 = to_snakecase(gpd.read_file("https://maps.scag.ca.gov/scaggis/rest/services/OpenData/TPA_RTP2020_Plan2045_boundary_scag/MapServer/0/query?outFields=*&where=1%3D1&f=geojson")) 

In [15]:
#scag2.shape

In [16]:
#scag2.plot()

### Sacramento Area Council of Governments (SACOG)
* https://www.sacog.org/sites/main/files/file-attachments/2020_mtp-scs.pdf?1580330993

In [17]:
#sacog = to_snakecase(gpd.read_file("https://services.sacog.org/hosting/rest/services/Hosted/2040_Major_Stops/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"))

### Lost

In [18]:
lost = harmonization_utils.load_lost()

In [19]:
lost.sample()

,measure,local_returns,project_category,location,project_title,cost__in_millions_,estimated_lost_funds,estimated_federal_funds,estimated_state_funds,estimated_local_funds,estimated_other_funds,project_description,agency,county,city,notes
194,imperial_d_2008,Yes,NaN,NaN,o st from barioni blvd to 13 st,0,0,0.00,0.00,0,0.00,construction/repair/maintenance,NaN,imperial,City of Westmorland,NaN


In [20]:
def harmonize_lost():
    df = harmonization_utils.load_lost()
    df = harmonizing_lrtp(
        df,
        project_name_col="project_title",
        project_description_col="project_description",
        project_category_col="project_category",
        project_cost_col="cost__in_millions_",
        location_col="",
        county_col="county",
        city_col="city",
        project_year_col="",
        program="",
        phase_col="phase",
        data_source="LOST",
        note_cols=["measure", 'estimated_lost_funds',
       'estimated_federal_funds', 'estimated_state_funds',
       'estimated_local_funds', 'estimated_other_funds',"notes"],
        cost_in_millions=False,
    )
    
    
    return df

### Project Cost Division

In [32]:
def correct_project_cost(df, project_title_col:str, project_total_cost:str):
    """
    For some datasets, the same project 
    (as determined by the same project name, cost,
    and source) is split across multiple rows. 

    Ex: A project costs $500 million and is 
    split on 5 rows by phase/location. Each row still lists 
    the total  cost as $500 million, which is not accurate.
    This function will recalculate each of the row to list
    $100 mil as the total project cost
    """
    # Create a unique identifier
    df['unique_identifier'] =  df[project_title_col] + df[project_total_cost].astype(str)
    
    # Create count for each project
    df['how_many_times_same_proj_appears'] = df.groupby('unique_identifier').cumcount()+1
    
    # Find the total number of times a project title-cost appears. 
    # Sort by descending and keep only the row with the highest level
    df2 = (df[[project_title_col,'how_many_times_same_proj_appears',project_total_cost, 'unique_identifier']]
            .sort_values([project_title_col, 'how_many_times_same_proj_appears'], ascending = False)
            .drop_duplicates(subset = ['unique_identifier'])
           ) 
    # Create new funding estimate
    df2['new_proj_cost'] = df2[project_total_cost] / df2["how_many_times_same_proj_appears"]
    
    # Drop some columns
    df2 = df2.drop(columns = [project_title_col,project_total_cost,'how_many_times_same_proj_appears'])
    
    # Merge
    m1 = pd.merge(df, df2, how = "inner", on = "unique_identifier")
    
    # Clean up
    m1 = m1.drop(columns = ['unique_identifier', 'how_many_times_same_proj_appears', project_total_cost])
    m1['total_project_cost_note'] = "This is an estimate of how much the project cost, estimated by dividing the total project cost by how many times the project appears in the dataset."
    
    # Replace project cost
    m1 = m1.rename(columns = {'new_proj_cost':'total_project_cost'})
    return m1

In [26]:
mtc = harmonize_mtc()

/opt/conda/lib/python3.9/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
/tmp/ipykernel_749/1718985932.py:36: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.


In [34]:
mtc2 = correct_project_cost(mtc,'project_title', 'total_project_cost')

In [31]:
mtc2.columns

Index(['project_title', 'lead_agency', 'project_year', 'project_category',
       'grant_program', 'project_description', 'phase', 'geometry', 'city',
       'county', 'data_source', 'notes', 'new_proj_cost',
       'total_project_cost_note'],
      dtype='object')

In [39]:
mtc2.drop(columns= ['geometry'])[['project_title','total_project_cost']].sort_values(['project_title']).head(10)

,project_title,total_project_cost
16,511 Bay Area Program | Regional,340.00
17,All Electronic Tolling Program | Regional,200.00
245,Automated People Mover | Service Expansion | VTA | Mineta San Jose International Airport Connector Automated People Mover,500.00
115,BRT | Modernization | AC Transit | 23rd St,259.00
143,BRT | Modernization | AC Transit | San Pablo Ave,300.00
212,BRT | Modernization | SFMTA | Geary Blvd,388.50
213,BRT | Modernization | SFMTA | Geary Blvd,388.50
214,BRT | Modernization | SFMTA | Geneva Ave/Harney Way,68.00
223,BRT | Modernization | SFMTA | Van Ness Ave,225.00
227,BRT | Modernization | SamTrans | El Camino Real,352.00


### Concat

In [ ]:
def all_mpo():
    tahoe = harmonize_tahoe()
    sandag = harmonize_sandag()
    mtc = harmonize_mtc()
    lost = harmonize_lost()
    df = pd.concat([tahoe, mtc, sandag, lost])

    return df

In [ ]:
all_mpo_df = all_mpo()

In [ ]:
all_mpo_df.groupby(["grant_program"]).size()

In [ ]:
congest_keywords = [
    "congestion pricing",
    "variablly priced",
    "variable tolls",
    "express lane",
    "value",
    "rush hour",
    "cordon",
    "dynamic pricing",
    "dynmically priced",
    "high occupancy toll",
    "high-occupancy"
    "mobility pricing",
    "occupancy",
    "toll lane",
    "performance pricing",
    "peak travel",
    "managed",
    "tollway",
    "transponder",
    "express toll",
    "fixed"
]

In [ ]:
def lower_case(df, columns_to_search: list):
    """
    Lowercase certain columns
    """
    new_columns = []
    for i in columns_to_search:
        df[f"lower_case_{i}"] = (df[i].str.lower().fillna("none")
                                 .str.replace("-","")
                                 .str.replace(".","")
                                 .str.replace(":","")
                                )
        new_columns.append(f"lower_case_{i}")

    return df,new_columns

In [ ]:
def find_keywords(df, columns_to_search:list, keywords_search:list):
    df2, lower_case_cols_list = lower_case(df, columns_to_search)
    
    keywords_search = f"({'|'.join(keywords_search)})"
    
    for i in lower_case_cols_list:
        df2[f"{i}_keyword_search"] = df2[i].str.extract(keywords_search).fillna("not congestion")
    
    return df2

In [ ]:
# test = find_keywords(all_mpo_df,  ["project_title", "project_description", "notes"], congest_keywords)

In [ ]:
def filter_cordon(df, columns_to_search:list, keywords_search:list, save_to_gcs: bool = False):

    # Filter out for Cordon 
    df = find_keywords(df, columns_to_search, keywords_search)
    df2 = (df[(df.lower_case_project_title_keyword_search != "not congestion")
        | (df.lower_case_project_description_keyword_search != "not congestion")]).reset_index(drop=True)
    
    # Delete out HOV
    projects_to_delete = ["SR 17 Corridor Congestion Relief in Los Gatos",
                         "Interstate 380 Congestion Improvements"]
    df2 = df2[~df2.project_title.isin(projects_to_delete)].reset_index(drop = True)

    # Drop invalid geometries
    gdf = df2[~df2.geometry.is_empty].reset_index(drop = True)
    gdf = gdf[gdf.geometry.is_valid].reset_index(drop = True)
    
    gdf = gdf.set_geometry("geometry")
    gdf = gdf.fillna(gdf.dtypes.replace({"float64": 0.0, "object": "None"}))

    # One version that's a df
    columns_to_drop = ['lower_case_project_title', 'lower_case_project_description']
    df2 = df2.drop(columns = columns_to_drop + ['geometry'])
    # df2 = df2.fillna(df.dtypes.replace({"float64": 0.0, "object": "None"}))
    
    if save_to_gcs:
        df2.to_excel(f"{harmonization_utils.GCS_FILE_PATH}LRTP/congestion_pricing_sample.xlsx", sheet_name="Sheet_name_1", index=False)
        gdf.to_file("./congestion_pricing_sample.geojson", driver="GeoJSON")
        
    return gdf, df2

In [ ]:
cordon_gdf, cordon_df = filter_cordon(all_mpo_df, ["project_title", "project_description",], congest_keywords)

In [ ]:
len(cordon_df.drop_duplicates(subset = ['project_title']))

In [ ]:
cordon_df.groupby(["grant_program"]).agg({"project_title":"nunique"})

In [ ]:
cordon_df.grant_program.value_counts()

In [ ]:
cordon_df.drop_duplicates(subset = ['total_project_cost','project_title',]).groupby(["grant_program"]).agg({'total_project_cost':'sum'})

In [ ]:
cols_to_drop = ['city','county', 'notes', 'phase']

In [ ]:
cordon_df.drop(columns =cols_to_drop).drop_duplicates(subset = ['project_title', 'project_description'])

In [ ]:
cordon_gdf.explore('project_title', cmap= 'tab20c', style_kwds={'weight':6}, legend = False)